# Preprocessing: LL17


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [2]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [1]:
## Prep paths ##

subject = 'LL17'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL17/ieeg/LL17_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL17/ieeg/LL17_raw_clean_ieeg.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.


In [4]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL17_channel_names.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bci_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)

# missed one empyt channel
filtered_clean_fif.info['bads'].append('CH72')
raw_clean_fif.info['bads'].append('CH72')


## Bipolar Rereferencing

In [5]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [6]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

LA1 LA2
LA2 LA3
LA3 LA4
LA4 LA5
LA5 LA6
LA6 LA7
LA7 LA8
LH1 LH2
LH2 LH3
LH3 LH4
LH4 LH5
LH5 LH6
LH7 noref
LH8 noref
LHP1 LHP2
LHP2 LHP3
LHP3 LHP4
LHP4 LHP5
LHP5 LHP6
LHP7 noref
LHP8 noref
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF10
LOF10 LOF11
LOF11 LOF12
RA1 RA2
RA2 RA3
RA3 RA4
RA4 RA5
RA5 RA6
RA6 RA7
RA7 RA8
RH1 RH2
RH2 RH3
RH3 RH4
RH4 RH5
RH5 RH6
RH6 RH7
RH7 RH8
RHP1 RHP2
RHP2 RHP3
RHP3 RHP4
RHP4 RHP5
RHP5 RHP6
RHP6 RHP7
RHP8 noref
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
ROF10 ROF11
ROF11 ROF12
CH72 noref
CH73 noref
CH74 noref
CH75 noref
CH76 noref
CH77 noref
CH78 noref
CH79 noref
CH80 noref
CH81 noref
CH82 noref
CH83 noref
CH84 noref
CH85 noref
CH86 noref
CH87 noref
CH88 noref
CH89 noref
CH90 noref
CH91 noref
CH92 noref
CH93 noref
CH94 noref
CH95 noref
CH96 noref
CH97 noref
CH98 noref
CH99 noref
CH100 noref
CH101 noref
CH102 noref
CH103 noref
CH104 noref
CH105 noref
CH106 noref
CH107 n

In [7]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 982099  =      0.000 ...  1918.162 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=59, n_times=982100
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/mne/io/reference.py:557: RuntimeWarning: invalid value encountered in matmul
  ref_data = multiplier @ inst._data


Added the following bipolar channels:
LA1-LA2, LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LA7-LA8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LOF10-LOF11, LOF11-LOF12, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP1-RHP2, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif


/tmp/ipykernel_77955/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif
[done]
Reading 0 ... 982099  =      0.000 ...  1918.162 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=59, n_times=982100
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Added the following bipolar channels:
LA1-LA2, LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LA7-LA8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LOF10-LOF11, LOF11-LOF12, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP1-RHP2, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12
Overwriting existing file.
Writ

/tmp/ipykernel_77955/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_raw_clean_data.fif
[done]


In [3]:
# # Visualize it #
# bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
# events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
# bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 59, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [4]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.


/tmp/ipykernel_78105/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


504 events found
Event IDs: [0 1]
Not setting metadata
252 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 8193 original time points ...
Loading data for 252 events and 8193 original time points ...


/tmp/ipykernel_78105/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Trial End

In [5]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.


/tmp/ipykernel_78105/749539899.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


504 events found
Event IDs: [0 1]
Not setting metadata
252 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 252 events and 5121 original time points ...


/tmp/ipykernel_78105/749539899.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Last Away

In [6]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,6,18,85068,0,1
1,7,19,89241,0,1
2,10,17,102886,0,1
3,11,15,108441,0,1
4,13,9,116428,0,1
...,...,...,...,...,...
186,243,20,954547,0,1
187,244,6,957056,0,1
188,245,4,959232,0,1
189,247,18,965836,0,1


In [7]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [8]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Not setting metadata
191 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 191 events and 5121 original time points ...


/tmp/ipykernel_78105/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78105/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [9]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,6,53401,0,1
1,1,3,59340,0,1
2,2,7,63180,0,1
3,3,5,72345,0,1
4,4,4,76492,0,1
...,...,...,...,...,...
227,246,2,961664,0,1
228,247,18,965196,0,1
229,248,5,968473,0,1
230,249,17,971212,0,1


In [10]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [11]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 4609 original time points ...
Loading data for 232 events and 4609 original time points ...


/tmp/ipykernel_78105/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78105/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [12]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,6,53376,0,1
1,1,3,59161,0,1
2,2,7,63052,0,1
3,3,5,72192,0,1
4,4,4,76313,0,1
...,...,...,...,...,...
240,246,2,961587,0,1
241,247,18,965120,0,1
242,248,5,968448,0,1
243,249,17,971008,0,1


In [13]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [14]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Not setting metadata
245 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 245 events and 5121 original time points ...


/tmp/ipykernel_78105/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78105/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [15]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,6,55014,0,1
1,1,3,59648,0,1
2,2,7,64512,0,1
3,3,5,73856,0,1
4,4,4,76800,0,1
...,...,...,...,...,...
81,237,5,934425,0,1
82,239,15,940646,0,1
83,246,2,962252,0,1
84,248,5,968857,0,1


In [16]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif...
    Range : 0 ... 982099 =      0.000 ...  1918.162 secs
Ready.
Not setting metadata
86 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 86 events and 5121 original time points ...


/tmp/ipykernel_78105/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78105/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL17/ieeg/LL17_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
